In [1]:
# set up matplotlib
%matplotlib widget
from matplotlib import pyplot as plt
plt.rcParams["figure.figsize"] = (8, 8)
plt.rcParams.update({"font.size": 14})

In [2]:
import numpy as np
import xarray as xr
from scipy.special import j0
from scipy.interpolate import RectBivariateSpline
from xbout import open_boutdataset

In [3]:
case = "1" # Case 1 is for anomalous density diffusion only.
filepath = "circle_nocut/BOUT.dmp.*.nc"
#filepath = "test" + str(case) + "/BOUT.dmp.0.nc"

ds = open_boutdataset(datapath=filepath, chunks={"t": 4})
dsn = xr.open_dataset("circle_nocut_lowestres.fci.nc")

# Use squeeze() to get rid of the y-dimension, which has length 1 usually unless turbulent.
ds = ds.squeeze(drop=True)
dsn = dsn.squeeze(drop=True)

for key,val in ds.items():
    print(key)

Read in:
<xbout.BoutDataset>
Contains:
<xarray.Dataset> Size: 46MB
Dimensions:             (x: 68, y: 1, z: 68, t: 201)
Coordinates:
    dx                  (x, y, z) float64 37kB dask.array<chunksize=(68, 1, 68), meta=np.ndarray>
    dy                  (x, y, z) float64 37kB dask.array<chunksize=(68, 1, 68), meta=np.ndarray>
    dz                  (x, y, z) float64 37kB dask.array<chunksize=(68, 1, 68), meta=np.ndarray>
  * t                   (t) float64 2kB 0.0 5e+05 1e+06 ... 9.95e+07 1e+08
  * x                   (x) int64 544B 0 1 2 3 4 5 6 7 ... 61 62 63 64 65 66 67
  * y                   (y) float64 8B 0.1963
  * z                   (z) float64 544B 0.007353 0.02206 ... 0.9779 0.9926
Data variables: (12/52)
    Bxy                 (x, y, z) float64 37kB dask.array<chunksize=(68, 1, 68), meta=np.ndarray>
    G1                  (x, y, z) float64 37kB dask.array<chunksize=(68, 1, 68), meta=np.ndarray>
    G2                  (x, y, z) float64 37kB dask.array<chunksize=(68, 1, 

/home/tirkas1/miniconda3/lib/python3.13/site-packages/xbout/load.py:341: UserWarning: No geometry type found, no physical coordinates will be added
  warn("No geometry type found, no physical coordinates will be added")


In [4]:
dx = ds["dx"].isel(z=0).values

# Get rid of existing "x" coordinate, which is just the index values.
#ds = ds.drop("x")

# Create a new coordinate, which is length in units of rho_s
ds = ds.assign_coords(x=np.arange(ds.sizes["x"])*dx)

#Store just initial values as well.
ds_initial = ds.isel(t=0)

In [5]:
dens = ds["Nh+"]
src = ds["SNh+"]
dens_true = dens.copy()

mask = dsn["in_mask"]
#vols = dsn["vol_frac"]
ghost_id = dsn["ghost_id"].values
#Get mask of ghost and inside cells.
ghost_mask = (ghost_id >= 0).astype(bool)
full_mask = ghost_mask | mask.astype(bool)
#For cut cells, need to ignore ghost cells with vol 1 because BOUT runs all cells and 1 outside to not div by 0...
#full_ghost_mask = ghost_mask & (vols >= 1).astype(bool)
#full_mask = full_mask & ~full_ghost_mask
#vols = np.where(full_mask, vols, 0)

gst_pts = dsn["ghost_pts"].values
bdy_pts = dsn["bndry_pts"].values
img_pts = dsn["image_pts"].values
normals = dsn["normals"].values
is_plasma = dsn["is_plasma"].values
Rarr = dsn["R"].values
Zarr = dsn["Z"].values

timestep_for_errs_early = 10
timestep_for_errs_late = 100
spl_e = RectBivariateSpline(Rarr[:,0], Zarr[0,:], dens.values[timestep_for_errs_early,:,:], kx=1, ky=1)
spl_l = RectBivariateSpline(Rarr[:,0], Zarr[0,:], dens.values[timestep_for_errs_late,:,:], kx=1, ky=1)

In [6]:
beta0 = 3.8317059702075125

x_unit = 0.00022846974366979959 #rho_s0
w_unit = 95788333.030660808    #Omega_ci
t_unit = 1/w_unit

nx = 64*int(case)
nz = nx + 4
dx = np.float64(1.0) / np.float64(nx)
dz = np.float64(1.0) / np.float64(nz)

# Centers in x; faces/nodes in z if that's your convention
x = (np.arange(nx, dtype=np.float64) + 0.5) * dx     # centered x
z = (np.arange(nz, dtype=np.float64)) * dz           # unshifted z

ghosts_lo_x = x[0]  - dx*np.arange(2, 0, -1)
ghosts_hi_x = x[-1] + dx*np.arange(1, 2+1)
x = np.concatenate((ghosts_lo_x, x, ghosts_hi_x))
xx,zz = np.meshgrid(x,z,indexing='ij')

R_min, R_max = dsn["R"].values[2,0], dsn["R"].values[-3,0]
Z_min, Z_max = dsn["Z"].values[0,0], dsn["Z"].values[0,-1]
dR = dsn["R"].values[1,0] - dsn["R"].values[0,0]
dZ = dsn["Z"].values[0,1] - dsn["Z"].values[0,0]
R0, Z0, a = 2.5, -0.022058823529411686, 1.0

x0 = (x[-1]+x[0])/2
z0 = (z[-1]+z[0])/2
print(x0)
print(z0)

# with:
D = 1
t64 = np.asarray(ds["t_array"].values, dtype=np.float64) # 1D time
t_norm = t64*t_unit

rho = np.hypot((xx - x0), (zz - z0)) #(R_max-R_min)*, (Z_max-Z_min)*
spatial = j0(beta0 * rho / (1/3))
temp    = np.exp(-D * (beta0 / a)**2 * t_norm)

dens_true = dens.copy()
dens_true[:] = 1 + spatial[np.newaxis,:,:] * temp[:,np.newaxis,np.newaxis]

0.5
0.4926470588235294


In [17]:

# Inputs assumed:
# bdy_pts: (Nb, 2) array, columns [R_b, Z_b] for boundary intercepts (BI)
# img_pts: (Nb, 2) array, columns [R_ip, Z_ip] for image points (inside fluid)
# dist_n:  (Nb,) array, distance along normal from BI to IP (physical coordinates)
# x_spl, z_spl: splines such that x_spl.ev(R,Z), z_spl.ev(R,Z) give computational coords
# x, z: 1D arrays of computational grid coords (same indexing as dens)
# dens: array with shape (nt, nx, nz) in (t,x,z) with indexing='ij'
# t_array: 1D array of physical times
# analytic_density(R, Z, t): function returning analytic density at given physical coords

def neumann_boundary_error_at_time(t_index):
    # --- 1. Physical boundary & image points ---
    Rb = bdy_pts[:, 0]
    Zb = bdy_pts[:, 1]
    Rip = img_pts[:, 0]
    Zip = img_pts[:, 1]

    # --- 2. Map to computational coordinates (x,z) ---
    spl_e = RectBivariateSpline(Rarr[:,0], Zarr[0,:], dens.values[k,:,:], kx=1, ky=1)
    n_num_b  = spl_e.ev(Rb,  Zb)
    n_num_ip  = spl_e.ev(Rip, Zip)

    # --- 4. Numerical normal derivative (one-sided) ---
    # du/dn ≈ (u(IP) - u(BI)) / dist_n
    du_dn_num = (n_num_ip - n_num_b) / np.hypot(normals[:,0], normals[:,1])

    # --- 5. Analytic normal derivative (same stencil) ---
    #t = t_array[t_index]

    #n_ana_b  = analytic_density(Rb,  Zb,  t)
    #n_ana_ip = analytic_density(Rip, Zip, t)

    spl_true_e = RectBivariateSpline(Rarr[:,0], Zarr[0,:], dens_true.values[k,:,:], kx=1, ky=1)
    n_num_true_b  = spl_true_e.ev(Rb,  Zb)
    n_num_true_ip  = spl_true_e.ev(Rip, Zip)
    du_dn_ana = (n_num_true_ip - n_num_true_b) / np.hypot(normals[:,0], normals[:,1])
    #du_dn_ana = 0 #(n_ana_ip - n_ana_b) / normals #TODO: Get BC condition?

    # If Neumann BC is zero-flux, du_dn_ana should be ~0; you can
    # just look at |du_dn_num|. Otherwise compare num vs ana:

    err = du_dn_num - du_dn_ana

    # L2 and Linf norms along the boundary
    L2_bnd   = np.sqrt(np.nanmean(err**2))
    Linf_bnd = np.nanmax(np.abs(err))

    # Relative norms (guard against zero analytic derivative)
    denom_L2   = max(np.nanmean(np.abs(du_dn_ana)), 1e-14)
    denom_Linf = max(np.nanmax(np.abs(du_dn_ana)), 1e-14)

    L2_rel   = L2_bnd   / denom_L2
    Linf_rel = Linf_bnd / denom_Linf

    return {
        "du_dn_num": du_dn_num,
        "du_dn_ana": du_dn_ana,
        "err": err,
        "L2_bnd": L2_bnd,
        "Linf_bnd": Linf_bnd,
        "L2_rel": L2_rel,
        "Linf_rel": Linf_rel,
    }

# Example usage:
k = 100  # some time index
res = neumann_boundary_error_at_time(k)
print("Boundary Neumann L2 error:", res["L2_bnd"])
print("Boundary Neumann L∞ error:", res["Linf_bnd"])


Boundary Neumann L2 error: 8.089241244790457e-07
Boundary Neumann L∞ error: 2.086292404834822e-06


In [ ]:
M0 = (dens.where(mask)*dR*dZ)[0,:,:].sum().values
M1 = (dens.where(mask)*dR*dZ)[10,:,:].sum().values
M2 = (dens.where(mask)*dR*dZ)[100,:,:].sum().values
M3 = (dens.where(mask)*dR*dZ)[-1,:,:].sum().values
print(M0)
print(M1)
print(M2)
print(M3)
print("ΔM =", M3 - M0)
print("relative change =", (M3 - M0)/M0)
assert np.isclose(M3, M0, rtol=1e-10, atol=1e-14)

In [ ]:
M0_true = (dens_true.where(mask)*dR*dZ)[0,:,:].sum().values
M1_true = (dens_true.where(mask)*dR*dZ)[10,:,:].sum().values
M2_true = (dens_true.where(mask)*dR*dZ)[100,:,:].sum().values
M3_true = (dens_true.where(mask)*dR*dZ)[-1,:,:].sum().values
print(M0_true)
print(M1_true)
print(M2_true)
print(M3_true)
print("ΔM =", M3_true - M0_true)
print("relative change =", (M3_true - M0_true)/M0_true)
#assert np.isclose(M3_true, M0_true, rtol=1e-10, atol=1e-14)

print("vol diff: ", M0_true - M0)

In [ ]:
dens_masked      = dens.values[0,:,:][ghost_mask]
dens_true_masked = dens_true.values[0,:,:][ghost_mask]

l2 = np.sqrt(np.mean((dens_masked - dens_true_masked)**2))
print("Boundary Linf norm: ", np.max(dens_masked-dens_true_masked))
print("Boundary L2 norm: ", l2)

dens_masked      = dens.values[10,:,:][ghost_mask]
dens_true_masked = dens_true.values[10,:,:][ghost_mask]

l2 = np.sqrt(np.mean((dens_masked - dens_true_masked)**2))
print("Boundary Linf norm: ", np.max(dens_masked-dens_true_masked))
print("Boundary L2 norm: ", l2)

dens_masked      = dens.values[100,:,:][ghost_mask]
dens_true_masked = dens_true.values[100,:,:][ghost_mask]

l2 = np.sqrt(np.mean((dens_masked - dens_true_masked)**2))
print("Boundary Linf norm: ", np.max(dens_masked-dens_true_masked))
print("Boundary L2 norm: ", l2)

In [ ]:
R,Z = dsn["R"].values[:,0], dsn["Z"].values[0,:]

x_edges = np.concatenate(([R[0] - 0.5*dR],
                          0.5*(R[:-1] + R[1:]),
                          [R[-1] + 0.5*dR]))
z_edges = np.concatenate(([Z[0] - 0.5*dZ],
                          0.5*(Z[:-1] + Z[1:]),
                          [Z[-1] + 0.5*dZ]))
xxe, zze = np.meshgrid(x_edges, z_edges, indexing='ij')

In [ ]:
plt.close('all')      # optional, but guarantees a clean slate for the whole kernel
fig, ax = plt.subplots()  # new figure/axes each run
ax.pcolormesh(xxe,zze,vols)
plt.show()

In [ ]:

dens_diff = np.abs(dens.where(mask)-dens_true.where(mask)) # np.abs(dens.where(mask) - dens_true.where(mask))
l2_err_t = np.sqrt(np.nanmean(dens_diff**2, axis=(1, 2)))

init_dens_diff = dens_diff[0,:,:]
mask2 = init_dens_diff.notnull() & (init_dens_diff >= 1e-10)
nerrs = int(mask2.sum().compute())

ii, jj = np.where(mask2)

if nerrs > 0:
    print(init_dens_diff.values[ii[0],jj[0]])
    print(dens.values[0,ii[0],jj[0]])
    print(dens_true.values[0,ii[0],jj[0]])

print(nerrs)

plt.close('all')      # optional, but guarantees a clean slate for the whole kernel
print(np.max(l2_err_t))
fig, ax = plt.subplots()  # new figure/axes each run
ax.plot(t_norm, l2_err_t)
ax.set_xlabel("Time [s]")
ax.set_ylabel("L2 error (RMS)")
ax.set_title("Time evolution of L2 error")
plt.show()


In [ ]:
#dens_diff0log = np.log(dens_diff0)

#dens_diff0.bout.animate2D(aspect='equal')
dens_diff.where(mask).bout.animate2D(aspect='equal')

In [ ]:
#src_true[:,:,:] = -D * lbda * np.asarray(spatial)[np.newaxis,:,:]*np.asarray(temp)[:,np.newaxis,np.newaxis]
#src_diff = src.where(mask)-src_true.where(mask)
src.where(mask).bout.animate2D(aspect='equal')

In [ ]:
no_mask = ~(mask.astype(bool))
src[:,2:-2,:].where(no_mask[2:-2,:]).bout.animate2D(aspect='equal')
#src.bout.animate2D(aspect='equal')

In [ ]:
#no_mask = ~(mask.astype(bool))
cut_mask = (vols < 1).astype(bool) & (vols > 0).astype(bool)
sim_mask = cut_mask.astype(bool) | mask.astype(bool)
#src[:,2:-2,:].where(~sim_mask[2:-2,:]).bout.animate2D(aspect='equal')
#src[:,2:-2,:].where(sim_mask[2:-2,:]).bout.animate2D(aspect='equal')
src[:,2:-2,:].where(cut_mask[2:-2,:] & (np.abs(src.values[0,2:-2,:]) < 5e-6)).bout.animate2D(aspect='equal')

In [ ]:
#no_mask = ~(mask.astype(bool))
cut_mask = (vols < 1).astype(bool) & (vols > 0).astype(bool)
sim_mask = cut_mask.astype(bool) | mask.astype(bool)
#src[:,2:-2,:].where(~sim_mask[2:-2,:]).bout.animate2D(aspect='equal')
#src[:,2:-2,:].where(sim_mask[2:-2,:]).bout.animate2D(aspect='equal')
diff = dens.copy()
diff[:-1,:,:] = dens_true.values[1:,:,:] - dens_true.values[:-1,:,:]
diff.where(cut_mask).bout.animate2D(aspect='equal')